In [9]:
# Import packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier

from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# Reading the Data

In [10]:
train_data = pd.read_csv("../Data/train_data.csv")
val_data = pd.read_csv("../Data/validation_data.csv")
test_data = pd.read_csv("../Data/test_data.csv")

train_data.sample(5)

,Unnamed: 0,text,class_label,text_preprocessed
1387,9177,Democrats want 'major role' for Sanders: Reute...,0,democrat want major role sander reutersipso po...
5649,16608,Candidate Replaced by H-1B: Americans Should H...,0,candid replac h1b american input outsourcinga ...
10988,23417,Chicago Community Organizers Mobilize Flash Mo...,1,chicago commun organ mobil flash mob shut trum...
23046,17373,BLACK REPUBLICAN AND BRILLIANT NEUROSURGEON AN...,1,black republican brilliant neurosurgeon announ...
2513,13561,BOOM! WATCH TREY GOWDY Scorch FBI Director Com...,1,boom watch trey gowdi scorch fbi director come...


In [11]:
X_train_text = train_data["text_preprocessed"].values
y_train = train_data["class_label"].values

X_val_text = val_data["text_preprocessed"].values
y_val = val_data["class_label"].values

X_test_text = test_data["text_preprocessed"].values
y_test = test_data["class_label"].values

# Baseline Model (Decision Tree)
## Using CountVectoriser with Bag of Words, Bigrams

In [12]:
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,1))
vectorizer.fit(X_train_text)

X_train = vectorizer.transform(X_train_text)
X_val = vectorizer.transform(X_val_text)
X_test = vectorizer.transform(X_test_text)

In [13]:
print("number of features used:", len(vectorizer.get_feature_names()))

number of features used: 238266


In [14]:
# initialise Decision Tree
clf = DecisionTreeClassifier(criterion='gini',random_state=0) 

# train model
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [15]:
y_val_pred = clf.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      6361
           1       0.96      0.96      0.96      6659

    accuracy                           0.96     13020
   macro avg       0.96      0.96      0.96     13020
weighted avg       0.96      0.96      0.96     13020



In [16]:
y_test_pred = clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      6361
           1       0.96      0.96      0.96      6660

    accuracy                           0.96     13021
   macro avg       0.96      0.96      0.96     13021
weighted avg       0.96      0.96      0.96     13021



In [17]:
# from sklearn.tree import export_graphviz
# import pydotplus
# from six import StringIO  
# from IPython.display import Image

# feature_cols = ['text_preprocessed']

# dot_data = StringIO()
# export_graphviz(clf, out_file = dot_data, 
#                       feature_names = feature_cols,  
#                      filled = True, rounded = True,  
#                     special_characters = True)

# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
# Image(graph.create_png())

In [24]:
# from sklearn.tree import plot_tree

# plot_tree(clf.fit(X_train, y_train))

In [18]:
# get predictions and compare results
predictions = clf.predict(X_test)
results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
results.sample(10)

,Actual,Predicted
7645,1,1
2189,0,0
8944,0,1
10625,1,1
5798,1,1
10583,0,0
1198,0,0
1715,1,1
6183,1,1
12854,1,1


In [19]:
# Accuracy
accuracy = accuracy_score(y_test,predictions)
print("Accuracy:",accuracy)

Accuracy: 0.9612164964288457


In [20]:
# Precision and Recall
precision = metrics.precision_score(y_test, predictions)
recall = metrics.recall_score(y_test, predictions)
f_measure = metrics.f1_score(y_test, predictions)
print("Precision:",precision)
print("Recall:",recall)
print("F-measure:",f_measure)

Precision: 0.9609105885876891
Recall: 0.9633633633633634
F-measure: 0.9621354127614905


## Using Tf-ldf and Unigrams

In [21]:
tfidf_params = {'unigram':(1,1), 'unigram and bigram': (1,2), 'bigram':(2,2)}

for ngram, values in tfidf_params.items():
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=values)
    tfidf_vectorizer.fit(X_train_text)

    X_train = tfidf_vectorizer.transform(X_train_text)
    X_val = tfidf_vectorizer.transform(X_val_text)
    X_test = tfidf_vectorizer.transform(X_test_text)

    print(f"Model with {ngram}")
    clf.fit(X_train, y_train)

    # Validation Data
    print("Testing using validation data:")    
    y_val_pred = clf.predict(X_val)
    print(classification_report(y_val, y_val_pred))
    print("------------------------------------------")

    # Test Data
    print("Testing using test data:")
    y_test_pred = clf.predict(X_test)
    print(classification_report(y_test, y_test_pred))
    print("------------------------------------------")
    print("------------------------------------------")

Model with unigram
Testing using validation data:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      6361
           1       0.95      0.96      0.95      6659

    accuracy                           0.95     13020
   macro avg       0.95      0.95      0.95     13020
weighted avg       0.95      0.95      0.95     13020

------------------------------------------
Testing using test data:
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      6361
           1       0.96      0.96      0.96      6660

    accuracy                           0.96     13021
   macro avg       0.96      0.96      0.96     13021
weighted avg       0.96      0.96      0.96     13021

------------------------------------------
------------------------------------------
Model with unigram and bigram
Testing using validation data:
              precision    recall  f1-score   support

           0       0.95   